In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [2]:
train_df = pd.read_json('train.json')
test_df = pd.read_json('test.json')

### Feature Engineering:

In [3]:
# dropping irrelevant features
train_df.drop(['bathrooms', 'bedrooms', 'street_address', 'display_address'], axis=1, inplace=True)
test_df.drop(['bathrooms', 'bedrooms', 'street_address', 'display_address'], axis=1, inplace=True)

In [4]:
# convert the created column to datetime object so as to extract more features 
train_df["created"] = pd.to_datetime(train_df["created"])
test_df["created"] = pd.to_datetime(test_df["created"])
train_df.head()

,building_id,created,description,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price
10,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000
10000,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465
100004,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...","[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850
100007,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275
100013,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350


In [5]:
# creating a created_hour feature
train_df["created_hour"] = train_df["created"].dt.hour
test_df["created_hour"] = test_df["created"].dt.hour

# and a created_day feature
train_df["created_day"] = train_df["created"].dt.day
test_df["created_day"] = test_df["created"].dt.day

In [6]:
# creating a column for the number of features of each house
train_df["num_features"] = train_df["features"].apply(len)
test_df["num_features"] = test_df["features"].apply(len)

In [7]:
# creating a column for the number of photos of each house
train_df["num_photos"] = train_df["photos"].apply(len)
test_df["num_photos"] = test_df["photos"].apply(len)

In [8]:
# count of words present in description column 
train_df["num_desc_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))
test_df["num_desc_words"] = test_df["description"].apply(lambda x: len(x.split(" ")))

In [9]:
# deleting irrelevant features
train_df.drop(['photos', 'features', 'created', 'description'], axis=1, inplace=True)
test_df.drop(['photos', 'features', 'created', 'description'], axis=1, inplace=True)

In [10]:
# Let's split the data
y = train_df["interest_level"]
X = train_df.drop(['interest_level'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

#### Dealing with manager_id:

In [11]:
temp = pd.concat([X_train['manager_id'],pd.get_dummies(y_train)], axis = 1).groupby('manager_id').mean()
temp.columns = ['man_high_frac','man_low_frac', 'man_medium_frac']
temp['man_count'] = X_train.groupby('manager_id').count().iloc[:,1]
temp.tail(10)

,man_high_frac,man_low_frac,man_medium_frac,man_count
manager_id,,,,
ff0502ae23268fdc5706225a8735cce6,0.000000,1.000000,0.000000,2
ff45f97e4709a06df6ca905a77f69072,0.000000,0.000000,1.000000,1
ff50149b1ec6a10fbb386d761c7a4bc5,0.000000,1.000000,0.000000,1
ff52574a198d8a349f41d57bd374e8e9,0.714286,0.285714,0.000000,7
ff5a3509ebce238dfa4857839ddd24ce,0.000000,1.000000,0.000000,1
ff707b7f0a2ea94a26c784f06c9fec69,0.250000,0.500000,0.250000,4
ff810b43aecfacd00f20a2912ff1e288,0.000000,1.000000,0.000000,1
ff8f1175d95c5a866ef73399cd54a1a1,0.000000,1.000000,0.000000,3
ffd3c60281949a5cd806030203a79558,0.250000,0.750000,0.000000,4


In [12]:
# compute skill
temp['manager_skill'] = temp['man_high_frac']*2 + temp['man_medium_frac']

# get ixes for unranked managers...
unranked_managers_ixes = temp['man_count']<20

# ... and ranked ones
ranked_managers_ixes = ~unranked_managers_ixes

In [13]:
# compute mean values from ranked managers and assign them to unranked ones
mean_values = temp.loc[ranked_managers_ixes, ['man_high_frac','man_low_frac', 'man_medium_frac','manager_skill']].mean()
print(mean_values)

#temp.loc[unranked_managers_ixes,['high_frac','low_frac', 'medium_frac','manager_skill']] = mean_values.values
temp.tail(10)

man_high_frac      0.081139
man_low_frac       0.668020
man_medium_frac    0.250841
manager_skill      0.413119
dtype: float64


,man_high_frac,man_low_frac,man_medium_frac,man_count,manager_skill
manager_id,,,,,
ff0502ae23268fdc5706225a8735cce6,0.000000,1.000000,0.000000,2,0.000000
ff45f97e4709a06df6ca905a77f69072,0.000000,0.000000,1.000000,1,1.000000
ff50149b1ec6a10fbb386d761c7a4bc5,0.000000,1.000000,0.000000,1,0.000000
ff52574a198d8a349f41d57bd374e8e9,0.714286,0.285714,0.000000,7,1.428571
ff5a3509ebce238dfa4857839ddd24ce,0.000000,1.000000,0.000000,1,0.000000
ff707b7f0a2ea94a26c784f06c9fec69,0.250000,0.500000,0.250000,4,0.750000
ff810b43aecfacd00f20a2912ff1e288,0.000000,1.000000,0.000000,1,0.000000
ff8f1175d95c5a866ef73399cd54a1a1,0.000000,1.000000,0.000000,3,0.000000
ffd3c60281949a5cd806030203a79558,0.250000,0.750000,0.000000,4,0.500000


X_train = X_train.merge(temp.reset_index(),how='left', left_on='manager_id', right_on='manager_id')
X_train.head()

In [14]:
# add the features computed on the training dataset to the validation dataset
X_test = X_test.merge(temp.reset_index(),how='left', left_on='manager_id', right_on='manager_id')
new_manager_ixes = X_test['man_high_frac'].isnull()
X_test.loc[new_manager_ixes,['man_high_frac','man_low_frac', 'man_medium_frac','manager_skill']] = mean_values.values
X_test.head()

,building_id,latitude,listing_id,longitude,manager_id,price,created_hour,created_day,num_features,num_photos,num_desc_words,man_high_frac,man_low_frac,man_medium_frac,man_count,manager_skill
0,7499d2f4f119b2dd7368bd44be6b3160,40.7749,6986987,-73.9550,624c1fbd75e5f99e6a7164cce1b1b8a4,2050,14,9,0,2,8,0.233333,0.400000,0.366667,30.0,0.833333
1,ce6d18bf3238e668b2bf23f4110b7b67,40.7081,7141348,-74.0065,48da4647746078e946b008a561123653,3350,4,11,8,8,109,0.000000,0.887097,0.112903,62.0,0.112903
2,de44c85daef7c8136d27630192ca1db8,40.7221,7105247,-73.9872,cbcd513212d592e6c717d27be14f46ac,4200,5,3,11,5,134,0.032258,0.548387,0.419355,31.0,0.483871
3,0a7a527bbc21f14dda39af5085bc6bb2,40.8003,7222625,-73.9618,3ab7c26842f970df4f7dc555dffc387d,3200,22,27,0,7,24,0.000000,0.888889,0.111111,9.0,0.111111
4,63b74a7a833472152244b76bf6093ced,40.7703,6845260,-73.9592,cb87dadbca78fad02b388dc9e8f25a5b,2100,6,8,5,3,107,0.143969,0.346304,0.509728,257.0,0.797665


In [15]:
# add the features computed on the training dataset to the validation dataset
test_df = test_df.merge(temp.reset_index(),how='left', left_on='manager_id', right_on='manager_id')
new_manager_ixes = test_df['man_high_frac'].isnull()
test_df.loc[new_manager_ixes,['man_high_frac','man_low_frac', 'man_medium_frac','manager_skill']] = mean_values.values
test_df.head()

,building_id,latitude,listing_id,longitude,manager_id,price,created_hour,created_day,num_features,num_photos,num_desc_words,man_high_frac,man_low_frac,man_medium_frac,man_count,manager_skill
0,79780be1514f645d7e6be99a3de696c5,40.7185,7142618,-73.9865,b1b1852c416d78d7765d746cb1b8921f,2950,5,11,6,8,78,0.250000,0.375000,0.375000,16.0,0.875000
1,0,40.7278,7210040,-74.0000,d0b5648017832b2427eeb9956d966a14,2850,6,24,3,3,35,0.000000,1.000000,0.000000,4.0,0.000000
2,3dbbb69fd52e0d25131aa1cd459c87eb,40.7306,7103890,-73.9890,9ca6f3baa475c37a3b3521a394d65467,3758,4,3,3,6,333,0.000000,0.000000,1.000000,1.0,1.000000
3,783d21d013a7e655bddc4ed0d461cc5e,40.7109,7143442,-73.9571,0b9d5db96db8472d7aeb67c67338c4d2,3300,6,11,10,6,204,0.368421,0.289474,0.342105,38.0,1.078947
4,6134e7c4dd1a98d9aee36623c9872b49,40.7650,6860601,-73.9845,b5eda0eb31b042ce2124fd9e9fcfce2f,4900,5,12,14,7,174,0.106383,0.489362,0.404255,47.0,0.617021


#### Dealing with building_id in a similar way as manager_id:

In [16]:
bld_train_list = train_df.building_id.unique()
bld_test_list = test_df.building_id.unique()
bld_list = np.concatenate((bld_train_list,bld_test_list), axis=0)
bld_list = list(set(bld_list))

In [17]:

df100 = train_df[['building_id','interest_level']]
interest_dummies = pd.get_dummies(df100['interest_level'])
df100 = pd.concat([df100,interest_dummies[['low','medium','high']]], axis = 1).drop('interest_level', axis = 1)
df100.head()

,building_id,low,medium,high
10,53a5b119ba8f7b61d4e010512e0dfc85,0,1,0
10000,c5c8a357cba207596b04d1afd1e4f130,1,0,0
100004,c3ba40552e2120b0acfc3cb5730bb2aa,0,0,1
100007,28d9ad350afeaab8027513a3e52ac8d5,1,0,0
100013,0,1,0,0


In [18]:
gby = pd.concat([df100.groupby('building_id').mean(),df100.groupby('building_id').count()], axis = 1).iloc[:,:-2]
gby.columns = ['bd_low_freq','bd_medium_freq','bd_high_freq','count']

In [19]:
gby['bd_avg_interest'] = gby['bd_medium_freq']*1 + gby['bd_high_freq']*2
gby.sort_values(by = 'count', ascending = False).head()

,bd_low_freq,bd_medium_freq,bd_high_freq,count,bd_avg_interest
building_id,,,,,
0,0.915762,0.060705,0.023534,8286,0.107772
96274288c84ddd7d5c5d8e425ee75027,0.650909,0.272727,0.076364,275,0.425455
11e1dec9d14b1a9e528386a2504b3afc,0.674419,0.241860,0.083721,215,0.409302
80a120d6bc3aba97f40fee8c2204524b,0.586854,0.328638,0.084507,213,0.497653
bb8658a3e432fb62a440615333376345,0.523585,0.353774,0.122642,212,0.599057


In [20]:
mean_values = gby[['bd_low_freq','bd_medium_freq','bd_high_freq','bd_avg_interest']].mean()
mean_values
# this is what I added

bd_low_freq        0.553652
bd_medium_freq     0.300489
bd_high_freq       0.145859
bd_avg_interest    0.592207
dtype: float64

In [21]:
# add the features computed on the training dataset to the validation dataset
X_test = X_test.merge(temp.reset_index(),how='left', left_on='building_id', right_on='building_id')
new_manager_ixes = X_test['man_high_frac'].isnull()
X_test.loc[new_manager_ixes,['man_high_frac','man_low_frac', 'man_medium_frac','manager_skill']] = mean_values.values
X_test.head()

KeyError: 'building_id'